In [2]:
import numpy as np
import plotly.graph_objects as go
from scipy.io import loadmat


In [3]:
data = loadmat('citys_data.mat')  # 请替换为实际文件路径
citys = data['citys']

In [18]:
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Tue Nov 05 15:05:01 2019',
 '__version__': '1.0',
 '__globals__': [],
 'citys': array([[1501, 2154],
        [4521, 2015],
        [4234, 1535],
        [2543, 3514],
        [2451, 2465],
        [3326, 1521],
        [1548, 2415],
        [3549, 1504],
        [4358, 1546],
        [4950,  600],
        [3549, 2546],
        [2647,  900],
        [4566, 1576],
        [1545, 1676],
        [1765,  900],
        [3456, 3544],
        [2151, 1542],
        [4061, 2248],
        [3155, 2647],
        [3888, 3551],
        [4025, 2555],
        [4263, 3544],
        [3429, 1564],
        [3254, 2654],
        [4558, 3554],
        [2441, 3201],
        [3541, 3240],
        [3140, 2456],
        [2545, 2446],
        [4545, 3456],
        [2647, 1999]], dtype=uint16),
 'D': array([[1.00000000e-04, 2.53894348e+03, 2.87380462e+03, 2.57527338e+03,
         2.31809944e+03, 2.15870795e+03, 2.21658408e+03, 3.17403970e+03,
   

# 加载城市数据

In [4]:
citys

array([[1501, 2154],
       [4521, 2015],
       [4234, 1535],
       [2543, 3514],
       [2451, 2465],
       [3326, 1521],
       [1548, 2415],
       [3549, 1504],
       [4358, 1546],
       [4950,  600],
       [3549, 2546],
       [2647,  900],
       [4566, 1576],
       [1545, 1676],
       [1765,  900],
       [3456, 3544],
       [2151, 1542],
       [4061, 2248],
       [3155, 2647],
       [3888, 3551],
       [4025, 2555],
       [4263, 3544],
       [3429, 1564],
       [3254, 2654],
       [4558, 3554],
       [2441, 3201],
       [3541, 3240],
       [3140, 2456],
       [2545, 2446],
       [4545, 3456],
       [2647, 1999]], dtype=uint16)

In [16]:
len(citys)

31

In [20]:
dstence = data['D']
len(dstence)

31

# 计算城市间的距离

In [22]:
D = data['D']

In [23]:
len(D)

31

# 初始化参数

In [24]:
m = 50  # 蚂蚁数量
alpha = 1  # 信息素重要程度因子
beta = 5  # 启发函数重要程度因子
rho = 0.2  # 信息素挥发因子
Q = 1  # 常系数
Eta = 1. / D  # 启发函数（适应性函数的倒数）
Tau = np.ones((n, n))  # 信息素矩阵
Table = np.zeros((m, n), dtype=int)  # 路径记录表
iter_max = 150  # 最大迭代次数
Route_best = np.zeros((iter_max, n), dtype=int)  # 各代最佳路径
Length_best = np.zeros(iter_max)  # 各代最佳路径的长度
Length_ave = np.zeros(iter_max)  # 各代路径的平均长度

# 迭代寻找最佳路径

In [25]:
np.random.seed(42)  # 为了结果的可重复性
shortest_paths_every_30_iters = []  # 保存每30次迭代的最短路径信息
iter_intervals = 30  # 每30次迭代绘制一次

for iter in range(iter_max):
    # 随机产生各个蚂蚁的起点城市
    start = np.random.choice(n, m, replace=True)
    Table[:, 0] = start
    for i in range(m):
        for j in range(1, n):
            tabu = Table[i, :j]  # 已访问的城市集合
            allow = np.setdiff1d(np.arange(n), tabu)  # 待访问的城市集合
            P = (Tau[tabu[-1], allow] ** alpha) * (Eta[tabu[-1], allow] ** beta)
            P /= np.sum(P)
            next_city = np.random.choice(allow, p=P)
            Table[i, j] = next_city

    # 计算路径长度
    Length = np.zeros(m)
    for i in range(m):
        route = Table[i]
        Length[i] = np.sum(D[route[:-1], route[1:]]) + D[route[-1], route[0]]

    # 更新最短路径和平均路径长度
    if iter == 0 or np.min(Length) < Length_best[iter - 1]:
        Length_best[iter] = np.min(Length)
        Route_best[iter] = Table[np.argmin(Length)]
    else:
        Length_best[iter] = Length_best[iter - 1]
        Route_best[iter] = Route_best[iter - 1]
    Length_ave[iter] = np.mean(Length)

    # 更新信息素
    Delta_Tau = np.zeros((n, n))
    for i in range(m):
        route = Table[i]
        for j in range(1, n):
            Delta_Tau[route[j-1], route[j]] += Q / Length[i]
        Delta_Tau[route[-1], route[0]] += Q / Length[i]  # 封闭路径
    Tau = (1 - rho) * Tau + Delta_Tau

    # 每30次迭代后保存当前最短路径和迭代次数
    if (iter + 1) % iter_intervals == 0 or iter == iter_max - 1:
        shortest_paths_every_30_iters.append((iter + 1, Route_best[iter], Length_best[iter]))

# 这里我们假设之后会有代码来绘制保存的路径
for iter, route, length in shortest_paths_every_30_iters:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=citys[route, 0], y=citys[route, 1], mode='markers+lines', name=f'Iter: {iter} Length: {length}'))
    fig.update_layout(title=f'蚁群算法优化路径 {iter} 迭代',
                      xaxis_title='城市位置横坐标',
                      yaxis_title='城市位置纵坐标')
    fig.show()  # 在本地环境中解除注释以显示图形

# 显示结果

In [26]:
shortest_length = np.min(Length_best)
shortest_route = Route_best[np.argmin(Length_best)]

print(f'最短距离: {shortest_length}')
print(f'最短路径: {shortest_route + 1}')  # 转换为1-based索引以匹配MATLAB输出

最短距离: 15601.919532918735
最短路径: [ 1 15 14 12 13 11 23 16  5  6  7  2  4  8  9 10  3 18 17 19 24 25 20 21
 22 26 28 27 30 31 29]


In [29]:
shortest_route_1based = shortest_route + 1  # 将索引转换为1-based以匹配MATLAB

# 创建图形对象
fig = go.Figure()

# 添加城市点
fig.add_trace(go.Scatter(
    x=citys[shortest_route, 0],
    y=citys[shortest_route, 1],
    mode='markers+text',
    name='Cities',
    text=[str(idx) for idx in shortest_route_1based],  # 添加城市索引
    textposition="bottom center",
    marker=dict(size=8, color='blue')
))

# 添加路径线
fig.add_trace(go.Scatter(
    x=np.append(citys[shortest_route, 0], citys[shortest_route[0], 0]),
    y=np.append(citys[shortest_route, 1], citys[shortest_route[0], 1]),
    mode='lines',
    name='Path',
    line=dict(color='red')
))

# 标记起点
fig.add_trace(go.Scatter(
    x=[citys[shortest_route[0], 0]],
    y=[citys[shortest_route[0], 1]],
    mode='markers+text',
    name='Start',
    text='Start',
    textposition="bottom center",
    marker=dict(size=10, color='green')
))

# 更新布局
fig.update_layout(
    title=f'蚁群算法优化路径(最短距离: {shortest_length:.2f})',
    xaxis_title='城市位置横坐标',
    yaxis_title='城市位置纵坐标',
    showlegend=False
)

In [28]:
# 创建折线图对象
fig = go.Figure()

# 添加最短路径长度的折线图
fig.add_trace(go.Scatter(x=np.arange(1, iter_max + 1), y=Length_best,
                         mode='lines+markers', name='最短路径长度',
                         line=dict(color='blue')))

# 添加平均路径长度的折线图
fig.add_trace(go.Scatter(x=np.arange(1, iter_max + 1), y=Length_ave,
                         mode='lines+markers', name='平均路径长度',
                         line=dict(color='red')))

# 更新布局
fig.update_layout(title='每次迭代最短路径和平均路径的折线图',
                  xaxis_title='迭代次数',
                  yaxis_title='路径长度',
                  legend_title='路径类型')

# 显示图形
fig.show()